# Feature engineering with Featuretools

## Setup

In [41]:
import featuretools as ft
import pandas as pd

## Data import

In [10]:
household_degrees = pd.read_excel('data/2023-06-14 Household degrees.xlsx')

In [30]:
household_degrees.dtypes

HOUSEHOLD_ID                            int64
HOUSEHOLD_RPT_NAME                     object
DEGREES_CONCAT                         object
HOUSEHOLD_SPOUSE_ID                    object
SPOUSE_DEGREES_CONCAT                  object
HOUSEHOLD_RECORD                     category
PERSON_OR_ORG                        category
HOUSEHOLD_KSM_YEAR                      int64
FIRST_KSM_GRAD_DT              datetime64[ns]
RECORD_STATUS_CODE                   category
STATUS_CHANGE_DATE             datetime64[ns]
ENTITY_DEATH_DT                datetime64[ns]
HOUSEHOLD_MASTERS_YEAR                float64
HOUSEHOLD_LAST_MASTERS_YEAR           float64
HOUSEHOLD_PROGRAM                    category
HOUSEHOLD_PROGRAM_GROUP              category
dtype: object

In [47]:
# Function to convert columns to factor or other datatype
def convert_columns_to_type(df, colnames, dtype = 'category'):
    for col in colnames:
        df[col] = df[col].astype(dtype)

# Function to convert year number to date (default: end of fiscal year date)
def year_to_fiscal_date(df, col_in, col_out, year, month = 9, day = 1):
    return

In [29]:
convert_columns_to_type(
    household_degrees
    , ('PERSON_OR_ORG', 'HOUSEHOLD_RECORD', 'RECORD_STATUS_CODE', 'HOUSEHOLD_PROGRAM', 'HOUSEHOLD_PROGRAM_GROUP')
    , dtype = 'category'
)

In [35]:
household_degrees[['HOUSEHOLD_ID', 'HOUSEHOLD_RECORD', 'PERSON_OR_ORG', 'HOUSEHOLD_KSM_YEAR', 'RECORD_STATUS_CODE', 'HOUSEHOLD_PROGRAM']].head()

,HOUSEHOLD_ID,HOUSEHOLD_RECORD,PERSON_OR_ORG,HOUSEHOLD_KSM_YEAR,RECORD_STATUS_CODE,HOUSEHOLD_PROGRAM
0,1858,AL,P,1932,D,FT-CB
1,1869,AL,P,1915,D,FT-CB
2,1879,AL,P,1929,D,FT-CB
3,1882,AL,P,1913,D,FT-CB
4,1900,AL,P,1911,D,FT-CB


In [48]:
household_degrees['HOUSEHOLD_KSM_DT'] = year_to_fiscal_date(household_degrees['HOUSEHOLD_KSM_YEAR'])

AssertionError: 